# WAMBS Python Tutorial

**Authors**: original R Tutorial created by Laurent Smeets and Rens van de Schoot, with updates by Duco Veen; Python translation by Florian Metwaly.

In this tutorial you follow the steps of the when-to-Worry-and-How-to-Avoid-the-Misuse-of-Bayesian-Statistics - checklist [(the WAMBS-checklist)](https://www.rensvandeschoot.com/wambs-checklist/).

## WAMBS Checklist

### *When to worry, and how to Avoid the Misuse of Bayesian Statistics*

**To be checked before estimating the model**

1. Do you understand the priors?

**To be checked after estimation but before inspecting model results**

2.    Does the trace-plot exhibit convergence?
3.  Does convergence remain after doubling the number of iterations?
4.   Does the posterior distribution histogram have enough information?
5.   Do the chains exhibit a strong degree of autocorrelation?
6.   Do the posterior distributions make substantive sense?

**Understanding the exact influence of the priors**

7. Do different specifications of the multivariate variance priors influence the results?
8.   Is there a notable effect of the prior when compared with non-informative priors?
9.   Are the results stable when conducting a sensitivity analysis?
10.   Is the Bayesian way of interpreting and reporting model results used?

## Example Data

The data we will be using for this exercise is based on a study about predicting PhD-delays ([Van de Schoot, Yerkes, Mouw and Sonneveld 2013](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0068839)).  The data can be downloaded [here](https://www.rensvandeschoot.com/wp-content/uploads/2018/10/phd-delays.csv). Among many other questions, the researchers asked the Ph.D. recipients how long it took them to finish their Ph.D. thesis (n=333). It appeared that Ph.D. recipients took an average of 59.8 months (five years and four months) to complete their Ph.D. trajectory. The variable *B3_difference_extra* measures the difference between planned and actual project time in months (mean=9.96, minimum=-31, maximum=91, sd=14.43). For more information on the sample, instruments, methodology and research context we refer the interested reader to the paper.

For the current exercise we are interested in the question whether age (M = 30.7, SD = 4.48, min-max = 26-69) of the Ph.D. recipients is related to a delay in their project.

The relation between completion time and age is expected to be non-linear. This might be due to that at a certain point in your life (i.e., mid thirties), family life takes up more of your time than when you are in your twenties or when you are older.

So, in our model the gap (*B3_difference_extra*) is the dependent variable and age (*E22_Age*) and age$^2$(*E22_Age_Squared *) are the predictors. The data can be found in the file <span style="color:red"> ` phd-delays.csv` </span>.



##### _**Question:** Write down the null and alternative hypotheses that represent this question. Which hypothesis do you deem more likely?_

<details>
<summary>Click to show result</summary>

$H_0:$ $age$ is not related to a delay in the PhD projects.

$H_1:$ $age$ is related to a delay in the PhD projects.

$H_0:$ $age^2$ is not related to a delay in the PhD projects.

$H_1:$ $age^2$ is related to a delay in the PhD projects.

</details>

## Preparation - Importing and Exploring Data

If you are using Google Colab, remove the # from the following line and run the chunk to install the `bambi` library:

In [ ]:
#!pip install bambi

In [ ]:
import bambi as bmb # BAyesian Model-Building Interface in Python
import arviz as az # Exploratory analysis of Bayesian models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # plotting

SEED = 1337 # set a seed to get consistent results

You can find the data in the file <span style="color:red"> ` phd-delays.csv` </span>, which contains all variables that you need for this analysis. Although it is a .csv-file, you can directly load it into Python using the following syntax:

In [ ]:
dataPHD = pd.read_csv("phd-delays.csv", sep=";")
dataPHD.columns = ["diff", "child", "sex", "age", "age2"]

Alternatively, you can directly download them from GitHub into your Python work space using the following command:

In [ ]:
dataPHD = pd.read_csv("https://raw.githubusercontent.com/UtrechtUniversity/BayesianEstimation/main/content/tuesday/phd-delays.csv", sep=";")
dataPHD.columns = ["diff", "child", "sex", "age", "age2"]

GitHub is a platform that allows researchers and developers to share code, software and research and to collaborate on projects (see https://github.com/)

Once you have loaded in your data, it is advisable to check whether your data import worked well. Therefore, first have a look at the summary statistics of your data. you can do this by using the  `DataFrame.describe()` function.

##### _**Question:** Have all your data been loaded in correctly? That is, do all data points substantively make sense? If you are unsure, go back to the .csv-file to inspect the raw data._

In [ ]:
dataPHD.describe()

_The descriptive statistics make sense:_

_diff: Mean (9.97), SE (0.79)_

_age: Mean (31.68), SE (0.38)_

_age2: Mean (1050.22), SE (35.97)_


##   **Step 1: Do you understand the priors?**


### 1.Do you understand the priors?


Before actually looking at the data we first need to think about the prior distributions and hyperparameters for our model. For the current model, there are four priors for the following four parameters:

- the intercept
- the two regression parameters ($\beta_1$ for the relation with AGE and $\beta_2$ for the relation with AGE2)
- the residual variance ($\sigma^2_{\epsilon}$)

We first need to determine which distribution to use for the priors. Let&#39;s use for the

- intercept a normal prior with $\mathcal{N}(\mu_0, \sigma^{2}_{0})$, where $\mu_0$ is the prior mean of the distribution and $\sigma^{2}_{0}$ is the variance parameter
- $\beta_1$ a normal prior with $\mathcal{N}(\mu_1, \sigma^{2}_{1})$
- $\beta_2$ a normal prior with $\mathcal{N}(\mu_2, \sigma^{2}_{2})$
- $\sigma^2_{\epsilon}$ an Inverse Gamma distribution with $IG(\kappa_0,\theta_0)$, where $\kappa_0$ is the shape parameter of the distribution and $\theta_0$ the rate parameter

Next, we need to specify actual values for the hyperparameters of the prior distributions. Let&#39;s say we found a previous study and based on this study the following hyperparameters can be specified:

- intercept $\sim \mathcal{N}(-35, 20)$
- $\beta_1 \sim \mathcal{N}(.8, 5)$
- $\beta_2 \sim \mathcal{N}(0, 10)$
- $\sigma^2_{\epsilon} \sim IG(.5, .5)$ This is an uninformative prior for the residual variance, which has been found to perform well in simulation studies.

It is a good idea to plot these distributions to see how they look. To do so, one easy way is to sample a lot of values from one of these distributions and make a density plot out of it, see the code below. Replace the 'XX' with the values of the hyperparameters. *Note*:The scale parameter uses the standard deviation, not the variance. Therefore we use `np.sqrt(XX)`.

<pre><code>
n = 100_000

prior_intercept = np.random.normal(loc=XX, scale=np.sqrt(XX), size=n)
effect_age = np.random.normal(loc=XX, scale=np.sqrt(XX), size=n)
effect_age2 = np.random.normal(loc=XX, scale=np.sqrt(XX), size=n)

fig, axs = plt.subplots(2, 2, figsize=(10, 8))

az.plot_dist(prior_intercept, ax=axs[0, 0])
axs[0, 0].set_title("Prior Intercept")

az.plot_dist(effect_age, ax=axs[0, 1])
axs[0, 1].set_title("Effect Age")

az.plot_dist(effect_age2, ax=axs[1, 0])
axs[1, 0].set_title("Effect Age²")

axs[1, 1].axis("off")

plt.tight_layout()
plt.show()
</code></pre>

<details>
<summary>Click to show result</summary>

You can copy-paste the code below in a new code block and run the block to see the result

```python
n = 100_000
prior_intercept = np.random.normal(loc=-35, scale=np.sqrt(20), size=n)
effect_age = np.random.normal(loc=0.8, scale=np.sqrt(5), size=n)
effect_age2 = np.random.normal(loc=0, scale=np.sqrt(10), size=n)

fig, axs = plt.subplots(2, 2, figsize=(10, 8))

az.plot_dist(prior_intercept, ax=axs[0, 0])
axs[0, 0].set_title("Prior Intercept")

az.plot_dist(effect_age, ax=axs[0, 1])
axs[0, 1].set_title("Effect Age")

az.plot_dist(effect_age2, ax=axs[1, 0])
axs[1, 0].set_title("Effect Age²")

axs[1, 1].axis("off")

plt.tight_layout()
plt.show()
```

</details>

We can also plot what the expected delay would be given these priors. With these priors the regression formula would be: $delay=-35+ .8*age + 0*age^2$. These are just the means of the priors and do not yet qualify the different levels of uncertainty. Replace the 'XX' in the following code with the prior means.

<pre><code>
years = np.arange(20, 81)  # 20 to 80 inclusive
delay = XX + XX * years + XX * years**2

plt.plot(years, delay, linestyle='-')
plt.xlabel("Years")
plt.ylabel("Delay")
plt.title("Delay vs Years")
plt.show()
</code></pre>

<details>
<summary>Click to show result</summary>

You can copy-paste the code below in a new code block and run the block to see the result

```python
years = np.arange(20, 81)  # 20 to 80 inclusive
delay = -35 + 0.8 * years + 0 * years**2

plt.plot(years, delay, linestyle='-')
plt.xlabel("Years")
plt.ylabel("Delay")
plt.title("Delay vs Years")
plt.show()
```

<\details>

## **Step 2: Run the model and check for convergence**


To run a multiple regression with `bambi`, you first specify the model, then fit the model and finally acquire the summary (similar to the frequentist model using  `lm()`). The model is specified as follows:


1.  A dependent variable we want to predict.
2.  A "~", that we use to indicate that we now provide the other variables of interest
    (comparable to the '=' of the regression equation).
3.  The different independent variables separated by the summation symbol '+'.
4.  Finally, we insert that the dependent variable has a variance and that we
    want an intercept.
5. We do set a seed to make the results exactly reproducible.
6. To specify priors, we use the `bmb.Prior` function. Be careful, `bambi` uses standard deviations instead of variances in the normal distribution. The standard deviations is the square root of the variance, so a variance of 5 corresponds to a standard deviation of 2.24 and a variance of 10 corresponds to a standard deviation of 3.16.


There are many other options we can select, such as the number of chains, how many iterations we want, and how long of a warm-up phase we want, but we will just use the defaults for now.

For more information on the basics of `bambi`, see the [website and vignettes](https://bambinos.github.io/bambi/).


### 2. Does the trace plot exhibit convergence?

First we run the analysis with only a short burn-in period of 250 samples and then we take another 250 samples.

The following code is how to specify the regression model:


In [ ]:
# Specify Priors
prior_age = bmb.Prior("Normal", mu=.8, sigma=2.24)
prior_age2 = bmb.Prior("Normal", mu=.0, sigma=3.16)
prior_int = bmb.Prior("Normal", mu=-35, sigma=4.47)
prior_sigma = bmb.Prior("InverseGamma", alpha=.5, beta=.5)

priors_inf = {"Intercept": prior_int, "age": prior_age, "age2": prior_age2, "sigma": prior_sigma}

In [ ]:
# Specify Model
model_few_samples = bmb.Model("diff ~ age + age2",
                  data=dataPHD,
                  priors=priors_inf,
                  center_predictors=False)

# Run Model
## tune sets the burn-in samples
## draws sets the post burn-in draws
fit_few_samples = model_few_samples.fit(random_seed=SEED,
                tune=250,
                draws=250)

Now we can plot the trace plots.

In [ ]:
# Extract the posterior samples
## NOTE: The burn in phase is automatically discarded by bambi, so those draws are not shown in the plots
samples = fit_few_samples.posterior

# Extract variable names
parameters = list(samples.keys())

# Plot each parameters trace plot
for param in parameters:
    values = samples[param]
    plt.figure(figsize=(10, 3))
    for chain in range(values.shape[0]):
        plt.plot(values[chain].values, label=f"Chain {chain}")
    plt.title(f"Trace plot for {param}")
    plt.xlabel("Iteration")
    plt.ylabel("Value")
    plt.legend()
    plt.tight_layout()
    plt.show()

Alternatively, you can simply make use of the `arviz` library to get the trace plots (and other convergence statistics).

In [ ]:
az.plot_trace(fit_few_samples)

It seems like the trace (caterpillar) plots are mostly converged into one each other (we ideally want one fat caterpillar), though the caterpillar could be 'fatter'. Additionally, we get the marginal posterior distribution for each chain, in which we can also see that the sampled posteriors differ slightly between the chains, indicating that the sampler hasn't fully converged and we want more samples.

We can check if the chains have reached convergence by having a look at the convergence diagnostics. Two of these diagnostics of interest include the Gelman and Rubin diagnostic and the Geweke diagnostic.

* The Gelman-Rubin Diagnostic shows the potential scale reduction factor (PSRF) values (using the  within and between chain variability), which all should be close to 1 ($\geq 1.01$). If they aren't close to 1, you should use more iterations. Note: The Gelman and Rubin diagnostic is also automatically given in the summary provided by `bambi` under the column `r_hat`.
* The Geweke diagnostic is not readily availabe in `bambi` but can be calculated manually (if you want; usually $\hat{R}$ suffices).

To obtain the Gelman and Rubin diagnostic use:

In [ ]:
az.rhat(fit_few_samples)

Additionally we can use rank plots. Rank plots are histograms of ranked posterior draws, plotted separately for each chain. The rank plots should show a uniform distribution.

In [ ]:
az.plot_rank(fit_few_samples)

These statistics confirm that the chains have not completely converged (although they are not too problematic either). Therefore (and for the sake of illustration), we run the same analysis with more samples.

In [ ]:
# Specify Model
model = bmb.Model("diff ~ age + age2",
                  data=dataPHD,
                  priors=priors_inf,
                  center_predictors=False)

# Run Model with more iterations
fit = model.fit(random_seed=SEED,
                tune=2000,
                draws=2000)

Obtain the trace plots again.

In [ ]:
az.plot_trace(fit)

Obtain the Gelman and Rubin diagnostic again.

In [ ]:
az.rhat(fit)

In [ ]:
az.plot_rank(fit)

Now we see that the Gelman and Rubin diagnostic (PRSF) is close to 1 for all parameters and the ranks look uniformly distributed.

### 3. Does convergence remain after doubling the number of iterations?

As is recommended in the WAMBS checklist, we double the amount of iterations to check for local convergence.

In [ ]:
# Specify Model
model_doubleiter = bmb.Model("diff ~ age + age2",
                             data=dataPHD,
                             priors=priors_inf,
                             center_predictors=False)

# Run Model with double more iterations
fit_doubleiter = model_doubleiter.fit(random_seed=SEED,
                tune=4000,
                draws=4000)

You should again have a look at the above-mentioned convergence statistics, but we can also compute the relative bias to inspect if doubling the number of iterations influences the posterior parameter estimates ($bias= 100*\frac{(model \; with \; double \; iteration \; - \; initial \; converged \; model )}{initial \; converged \; model}$). In order to preserve clarity we  just calculate the bias of the two regression coefficients.

You should combine the relative bias in combination with substantive knowledge about the metric of the parameter of interest to determine when levels of relative deviation are negligible or problematic. For example, with a regression coefficient of 0.001, a 5% relative deviation level might not be substantively relevant. However, with an intercept parameter of 50, a 10% relative deviation level might be quite meaningful. The specific level of relative deviation should be interpreted in the substantive context of the model. Some examples of interpretations are:

- if relative deviation is &lt; |5|%, then do not worry;
- if relative deviation &gt; |5|%, then rerun with 4x the number of iterations.

_**Question:** calculate the relative bias. Are you satisfied with number of iterations, or do you want to re-run the model with even more iterations?_


To get the relative bias, save the means of the regression coefficients (posterior means) and other parameters for the two different analyses and compute the bias.

In [ ]:
#@title Click here for the solution
# Get posterior summaries
summary_fit = az.summary(fit)
summary_fit_doubleiter = az.summary(fit_doubleiter)

# Extract posterior means
means_fit = summary_fit["mean"]
means_fit_doubleiter = summary_fit_doubleiter["mean"]

# Compute % change
percent_change = 100 * ((means_fit_doubleiter - means_fit) / means_fit)
percent_change_rounded = percent_change.round(4)

print(percent_change_rounded)

### 4.   Does the posterior distribution histogram have enough information?

By having a look at the posterior distribution histogram `az.plot_posterior(fit, kind="hist", bins=30)`, we can check if it contains enough information.

_**Question:** What can you conclude about the distribution histograms?_

In [ ]:
az.plot_posterior(fit, kind="hist", bins=30)

<details>
<summary>Click to show interpretation</summary>

_The histograms look smooth and have no gaps or other abnormalities. Based on this, adding more iterations is not necessary. However, if you are not satisfied, you can increase the number of iterations again. Posterior distributions do not have to be symmetrical, but in this example they seem to be._

</details>

If we compare this with histograms based on the first analysis (with very few iterations), this difference becomes clear:

In [ ]:
az.plot_posterior(fit_few_samples, kind = "hist", bins=30)

### 5.   Is the effective sample size sufficient or do the chains exhibit a strong degree of autocorrelation?

The effective sample size (ESS) is a measure of the number of independent samples in a Markov Chain Monte Carlo (MCMC) chain and reflects the efficiency of the algorithm.  It accounts for the autocorrelation in the chains, which can reduce the effective number of samples. A higher ESS indicates that the MCMC chain is more efficient and provides more reliable estimates. `bambi` gives a warning if the number of ESS is too small.

In [ ]:
az.ess(fit)

To investigate the level of autocorrelation use:

In [ ]:
az.plot_autocorr(fit)

_**Question:** What can you conclude about these autocorrelation plots?_

<details>
<summary>Click to show interpretation</summary>

These results show that autocorrelation is quite stong after a few lags. This means it is important to make sure we ran the analysis with a lot of samples, because with a high autocorrelation it will take longer until the whole parameter space has been identified. Historically, thinning has been used to reduce autocorrelation in MCMC chains, but this is no longer recommended. Instead, it is better to increase the number of iterations and warmup samples to ensure that the chains are well-mixed and that the effective sample size is sufficient. For more information on autocorrelation check this [paper](https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/j.2041-210X.2011.00131.x).

</details>


### 6.   Do the posterior distributions make substantive sense?

We plot the posterior distributions and see 1) if they are unimodel (one peak); 2) if they are clearly centered around one value; 3) if they give a realistic estimate; and 4) if they make substantive sense compared to our prior believes (priors). Here we plot the  posteriors of the regression coefficients. If you want you can also plot the mean and the 95% Posterior HPD Intervals.


In [ ]:
az.plot_posterior(fit, hdi_prob=0.95)

_**Question:** What is your conclusion; do the posterior distributions make sense?_


<details>
<summary>Click to show interpretation</summary>

_Yes, we see a clear negative intercept, which makes sense since a value of age = 0 for Ph.D is impossible. We also have have plausible ranges of values for the regression coefficients and a positive variance._

</details>


In addition, do the posterior predictive distributions make sense substantively and in light of the observed data? For this, we want to look at predictions on the response scale the model makes based on the posterior parameter estimates.

In [ ]:
idata = model.fit(draws=2000, tune=1000)
idata.extend(model.predict(idata, kind="response", inplace=False))

az.plot_ppc(idata)

_**Question:** Do the posterior predictive distributions make sense substantively and in light of the observed data?_

<details>
<summary>Click to show interpretation</summary>
This is often the tricky part. Clearly, we see that the model does not perfectly capture the shape of the observed data, which is not symmetrical. Although we could argue that the fit is good enough for our purposes, we could also investigate how to improve the fit by considering alternative likelihood functions, such as a shifted lognormal distribution, or setting a lower bound based on the minimum possible delay (e.g., 2 years, so 12 months). Do note that while these changes will most likely improve the posterior fit, because they better suit the data-generating process, they will also make interpretation more complex.
</details>

## **Step 3: Understanding the exact influence of the priors**

First we  check the results of the analysis with the priors we used so far.

In [ ]:
az.summary(fit)

### 7. Do different specifications of the variance priors influence the results?

So far we have used the $\sigma^2_{\epsilon} \sim IG(.5, .5)$ prior, but we can also use the $\sigma^2_{\epsilon} \sim IG(.01, .01)$ prior and see if doing so makes a difference. To quantify this difference we again calculate a relative bias.

_**Question:** Are the results robust for different specifications of the prior on the residual variance?_


| Parameters | Estimate with $\sigma^2_{\epsilon} \sim IG(.01, .01)$ | Estimate with $\sigma^2_{\epsilon} \sim IG(.5, .5)$ | Bias |
| --- | --- | --- | --- |
| Intercept | |  | |
| Age       | |  | |
| Age2      | |  | |
| Residual variance |  |  |  |


In [ ]:
# Specify Priors
prior_age = bmb.Prior("Normal", mu=.8, sigma=2.24)
prior_age2 = bmb.Prior("Normal", mu=.0, sigma=3.16)
prior_int = bmb.Prior("Normal", mu=-35, sigma=4.47)
prior_sigma = bmb.Prior("InverseGamma", alpha=.01, beta=.01)

priors_inf2 = {"Intercept": prior_int, "age": prior_age, "age2": prior_age2, "sigma": prior_sigma}

# Specify Model
model_difIG = bmb.Model("diff ~ age + age2",
                  data=dataPHD,
                  priors=priors_inf2,
                  center_predictors=False)

# Run Model
## tune sets the burn-in samples
## draws sets the post burn-in draws
fit_difIG = model_difIG.fit(random_seed=SEED,
                tune=2000,
                draws=4000)

In [ ]:
az.summary(fit_difIG)

In [ ]:
#@title Click to show the code

# Get posterior summary of the different IG prior model
summary_fit_difIG = az.summary(fit_difIG)

# Extract posterior mean
means_fit_difIG = summary_fit_difIG["mean"]

# Compute % change
percent_change_difIG = 100 * ((means_fit_difIG - means_fit) / means_fit)
percent_change_rounded_difIG = percent_change_difIG.round(4)

#print(percent_change_rounded_difIG)

tab = pd.DataFrame({
    "Parameters" : ["Intercept", "Age", "Age2", "Residual variance"],
    "Estimates with IG(0.5,0.5) prior" : means_fit.round(3),
    "Estimate with IG(0.01,0.01) prior" : means_fit_difIG.round(3),
    "Bias (%)" : percent_change_rounded_difIG.round(3)
})
tab.style.hide(axis="index").format({
    "Estimates with IG(0.5,0.5) prior": lambda x: f"{x:.3f}",
    "Estimate with IG(0.01,0.01) prior": lambda x: f"{x:.3f}",
    "Bias (%)": lambda x: f"{x:.3f}"})


<details>
<summary>Click to show interpretation</summary>

Yes, the results are robust, because there is only a really small amount of relative bias for the residual variance.

</details>

### 8.   Is there a notable effect of the prior when compared with non-informative priors?

The default `bambi` priors are non-informative, so we can run the analysis without any specified priors and compare them to the model we have run thus far, using the relative bias, to see if there is a large influence of the priors.


_**Question**: What is your conclusion about the influence of the priors on the posterior results?_

| Parameters | Estimates with default priors | Estimate with informative priors | Bias|
| --- | --- | --- | --- |
| Intercept |  |      |  |
| Age |  |  | |
| Age2 |  |  ||
| Residual variance | |  |  |

The default priors in `bambi` are internally scaled to be 'generic weakly informative'. From the manual we learn that:

*By default, Bambi will intelligently generate weakly informative priors for all model terms, by loosely scaling them to the observed data. Currently, Bambi uses a methodology very similar to the one described in the documentation of the R package rstanarm. While the default priors will behave well in most typical settings, there are many cases where an analyst will want to specify their own priors–and in general, when informative priors are available, it’s a good idea to use them.*

The details on this scaling can be found here: https://mc-stan.org/rstanarm/articles/priors.html#default-weakly-informative-prior-distributions-1

We can run the model without our priors and check if doing so strongly influences the results. With this fitted model, we can also see what the default priors look like in our case.

In [ ]:
# Specify Model
model_default = bmb.Model("diff ~ age + age2",
                  data=dataPHD,
                  center_predictors=False)

# Run Model
## tune sets the burn-in samples
## draws sets the post burn-in draws
fit_default = model_default.fit(random_seed=SEED,
                tune=2000,
                draws=2000)

To see the priors used in the model -- in this case the default priors -- we can use: `model.plot_priors()`

In [ ]:
model_default.plot_priors()

Or we simply print the model to see the priors that were used.

In [ ]:
model_default

In [ ]:
az.summary(fit_default)

_**Question**: What is your conclusion on the influence of the informative versus the default priors?_

In [ ]:
#@title Click to show the code

# Get posterior summary of the default prior model
summary_fit_default = az.summary(fit_default)

# Extract posterior mean
means_fit_default = summary_fit_default["mean"]

# Compute % change
percent_change_default = 100 * ((means_fit_default - means_fit) / means_fit)
percent_change_rounded_default = percent_change_default.round(4)

#print(percent_change_rounded_default)

tab = pd.DataFrame({
    "Parameters" : ["Intercept", "Age", "Age2", "Residual variance"],
    "Estimates with informative priors" : means_fit.round(3),
    "Estimate with default priors" : means_fit_default.round(3),
    "Bias (%)" : percent_change_rounded_default.round(3)
})
tab.style.hide(axis="index").format({
    "Estimates with informative priors": lambda x: f"{x:.3f}",
    "Estimate with default priors": lambda x: f"{x:.3f}",
    "Bias (%)": lambda x: f"{x:.3f}"})


<details>
<summary>Click to show interpretation</summary>

_The informative priors have quite some influence (up to 25%) on the posterior results of the intercept and regression coefficients. This is not a bad thing, just important to keep in mind._

</details>

_**Question**: Which results do you use to draw conclusion on?_

<details>
<summary>Click to show interpretation</summary>

_This really depends on where the priors come from. If for example your informative priors come from a reliable source, you should use them. The most important thing is that you choose your priors accurately, and have good arguments to use them. If not, you shouldn&#39;t use really informative priors and use the results based on the non-informative priors._

</details>

### 9.   Are the results stable from a sensitivity analysis?

If you still have time left, you can adjust the hyperparameters of the priors upward and downward and re-estimating the model with these varied priors to check for robustness.

From the original paper:

> "If informative or weakly-informative priors are used, then we suggest running a sensitivity analysis of these priors. When subjective priors are in place, then there might be a discrepancy between results using different subjective prior settings. A sensitivity analysis for priors would entail adjusting the entire prior distribution (i.e., using a completely different prior distribution than before) or adjusting hyperparameters upward and downward and re-estimating the model with these varied priors. Several different hyperparameter specifications can be made in a sensitivity analysis, and results obtained will point toward the impact of small fluctuations in hyperparameter values. [...] The purpose of this sensitivity analysis is to assess how much of an impact the location of the mean hyperparameter for the prior has on the posterior. [...] Upon receiving results from the sensitivity analysis, assess the impact that fluctuations in the hyperparameter values have on the substantive conclusions. Results may be stable across the sensitivity analysis, or they may be highly instable based on substantive conclusions. Whatever the finding, this information is important to report in the results and discussion sections of a paper. We should also reiterate here that original priors should not be modified, despite the results obtained."


For more information on this topic, please also refer to this [paper](http://psycnet.apa.org/record/2017-52406-001).

In addition to sensitivity analyses based on prior settings, you may also include sensitivity checks based on the model structure, such as using a different likelihood function (e.g., the shifted log-normal distribution) or the inclusion of additional predictors or interactions.  

### 10.   Is the Bayesian way of interpreting and reporting model results used?

For a summary on how to interpret and report models, please refer to https://www.rensvandeschoot.com/bayesian-analyses-where-to-start-and-what-to-report/

Aspects to report:

1. estimates + credible intervals: make use of the fact that you have actual indication of uncertainty of the model parameters (not just hypothetical). Also add plots to ease interpretability!
2. program and packages used
3. discussion of priors (justify choices, and report sensitivity)
4. discussion of settings (number of chains, number of interaction, warmup, seeds etc.)
5. discussion of sampling diagnostics (e.g., $\hat{R}$, effective sample size)
6. perhaps: model fit or model comparison metrics (e.g., loo, Bayes factors)


In [ ]:
az.summary(fit)

In the current model we see that:

*  The estimate for the intercept is -36.26 [-43.97;-28.45]
*  The estimate for the effect of $age$  is 2.14 [1.77;2.53]
*  The estimate for the effect of $age^2$  is -0.02 [-0.03;-0.02]


We can see that none of 95% Posterior HPD Intervals for these effects include zero, which means we are can be quite certain that all of the effects are different from 0.


We used the `bambi` package to fit Bayesian multilevel models in Python. The model was run with 4 chains, each with 4000 iterations and a warmup of 2000 iterations (total post-warmup $N=8000$). The model diagnostics indicated good convergence (largest $\hat{R}=1$ for the residual variance and sufficient effective sample sizes for all parameters. The smallest $\hat{N}_{\text{eff}} = 1234.0$ for the regression coefficient of age (ratio = 0.15), indicating that there is some autocorrelation in the chains.

We used an linear regression model with a Gaussian likelihood and informative priors derived from (fictitious) previous studies on the intercept $\sim \mathcal{N}(-35, 20)$, the coefficient of age in years $\beta_1 \sim \mathcal{N}(.8, 5)$, the coefficient of squared age $\beta_2 \sim \mathcal{N}(0, 10)$, as we expected a non-linear effect of age, and the residual variance $\sigma \sim IG(.5, .5)$. This is an uninformative prior for the residual variance, which has been found to perform well in simulation studies.
Sensitivity checks on the priors and model specification indicated that the conclusions are robust to different prior settings for the variance (i.e., $\sigma \sim IG(.01, .01)$. However, result do differ when using diffuse default priors, which results in a lower intercept and stronger effects of age, though the pattern remains the same.
  

Remember how we plotted the relation between delay and years based on the prior information? Now, do the same with the posterior estimates.


In [ ]:
years = np.arange(20, 81)  # 20 to 80 inclusive
delay = -35 + 2.13 * years - 0.02 * years**2

plt.plot(years, delay, linestyle='-')
plt.xlabel("Years")
plt.ylabel("Delay")
plt.title("Delay vs Years")
plt.show()

Or, more Bayesian by including the credible interval to show the uncertainty in the predicted values.

In [ ]:
age_range = np.linspace(20, 80, 100)
newdata = pd.DataFrame({
    "age": age_range,
    "age2": age_range**2
})

In [ ]:
idata_pp = model.predict(idata, data=newdata, kind="response", inplace=False)
idata_pp.posterior_predictive
yrep = idata_pp.posterior_predictive["diff"].values  # shape: (draws, observations)
yrep = yrep.reshape(-1, yrep.shape[-1])

In [ ]:

mean = yrep.mean(axis=0)
lower = np.percentile(yrep, 2.5, axis=0)
upper = np.percentile(yrep, 97.5, axis=0)

newdata["mean"] = mean
newdata["lower"] = lower
newdata["upper"] = upper


In [ ]:
plt.figure(figsize=(8, 6))

# Line for mean prediction
plt.plot(newdata["age"], newdata["mean"], color="blue", label="Mean Prediction")

# Shaded area for 95% credible interval
plt.fill_between(newdata["age"], newdata["lower"], newdata["upper"], color="blue", alpha=0.3)

# Reference line
plt.axhline(0, color="gray", linestyle="--")

# Labels and title
plt.title("Predicted delay of PhD degree by age", fontsize=14)
plt.xlabel("Age (years)", fontsize=12)
plt.ylabel("Delay (months)", fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

**References**


Depaoli, S., &amp; Van de Schoot, R. (2017). Improving transparency and replication in Bayesian statistics: The WAMBS-Checklist. _Psychological Methods_, _22_(2), 240.

Link, W. A., & Eaton, M. J. (2012). On thinning of chains in MCMC. _Methods in ecology and evolution_, _3_(1), 112-115.

van Erp, S., Mulder, J., & Oberski, D. L. (2017). Prior sensitivity analysis in default Bayesian structural equation modeling.

Van de Schoot, R., &amp; Depaoli, S. (2014). Bayesian analyses: Where to start and what to report. _European Health Psychologist_, _16_(2), 75-84.

## Original Computing Environment

In [ ]:
#!pip install session_info

In [ ]:
import session_info
session_info.show()